# Logistic Regression Refresher

Based on [this resource](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/)

#### Import Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

def probability(logodds):
    '''
    This function takes log odds from linear regression 
    and translates to probability.
    '''
    return np.exp(logodds)/(1 + np.exp(logodds))

In [ ]:
data = pd.read_csv("data/sample.csv")
data = sm.add_constant(data)
data.head()

#### Logistic regression with no predictor variables

Let’s start with the simplest logistic regression, a model without any predictor variables.  In an equation, we are modeling $logit(p)=\beta_0$

In [ ]:
lr = sm.Logit(data['hon'], data['const']).fit()
lr.summary()

In [ ]:
# # Formula notation, identical results
# lr = smf.logit('hon ~ 1', data=data).fit()
# lr.summary()

This means $log(p/(1-p)) = -1.12546$.  What is $p$ here?  It turns out that $p$ is the overall probability of being in honors class (hon = 1).  Let’s take a look at the frequency table for hon.

In [ ]:
data.hon.value_counts()

So $p = 49 / (151 + 49) =  .245$. The odds are $.245/(1-.245) = .3245$ and the log of the odds (logit) is $log(.3245) = -1.12546$.  In other words, the intercept from the model with no predictor variables is the estimated log odds of being in honors class for the whole population of interest.  We can also transform the log of the odds back to a probability: $p = exp(-1.12546)/(1+exp(-1.12546)) = .245$, if we like.

#### Logistic regression with a single dichotomous predictor variables

Now let’s go one step further by adding a binary predictor variable, female, to the model.  Writing it in an equation, the model describes the following linear relationship: $logit(p) = \beta_0 + \beta_1 * female$

In [ ]:
lr = sm.Logit(data['hon'], data[['female','const']]).fit()
lr.summary()

Before trying to interpret the two parameters estimated above, let’s take a look at the crosstab of the variable hon with female.

In [ ]:
pd.crosstab(data.hon, data.female)

In our dataset, what are the odds of a male being in the honors class and what are the odds of a female being in the honors class?  We can manually calculate these odds from the table: for males, the odds of being in the honors class are $(17/91)/(74/91) = 17/74 = .23$; and for females, the odds of being in the honors class are $(32/109)/(77/109) = 32/77 = .42$.  The ratio of the odds for female to the odds for male is $(32/77)/(17/74) = (32*74)/(77*17) = 1.809$.  So the odds for males are 17 to 74, the odds for females are 32 to 77, and the odds for female are about 81% higher than the odds for males.

Now we can relate the odds for males and females and the output from the logistic regression.  The intercept of -1.471 is the log odds for males since male is the reference group (`female = 0`).  Using the odds we calculated above for males, we can confirm this: $log(.23) = -1.47$.  The coefficient for female is the log of odds ratio between the female group and male group: $log(1.809) = .593$.  So we can get the odds ratio by exponentiating the coefficient for female. Most statistical packages display both the raw regression coefficients and the exponentiated coefficients for logistic regression models. It's a little bit manual in Python:


In [ ]:
np.exp(lr.params)

#### Logistic regression with an interaction term of two predictor variables

In all the previous examples, we have said that the regression coefficient of a variable corresponds to the change in log odds and its exponentiated form corresponds to the odds ratio.  This is only true when our model does not have any interaction terms.  When a model has interaction term(s) of two predictor variables, it attempts to describe how the effect of a predictor variable depends on the level/value of another predictor variable.  The interpretation of the regression coefficients become more involved.

Let’s take a simple example.

$$logit(p) = log(p/(1-p))= \beta_0 + \beta_1*female + \beta_2*math + \beta_3*female*math$$

In [ ]:
# Create Interaction
data['femalexmath'] = data['female'] * data['math']

lr = sm.Logit(data['hon'], data[['const', 'female', 'math', 'femalexmath']]).fit()
lr.summary()

In [ ]:
# # Formula notation, identical results. No need to create interaction var first
# lr = smf.logit(formula='hon ~ female + math + female:math', data=data).fit()
# lr.summary()

In the presence of interaction term of female by math, we can no longer talk about the effect of `female`, holding all other variables at certain value, since it does not make sense to fix `math` and `female:math` at certain value and still allow `female` change from 0 to 1!

In this simple example where we examine the interaction of a binary variable and a continuous variable, we can think that we actually have **two equations**: one for males and one for females.  For males `(female=0)`, the equation is simply

$$logit(p) = log(p/(1-p))= \beta_0 + \beta_2*math$$

For females, the equation is

$$logit(p) = log(p/(1-p))= (\beta_0 + \beta_1) + (\beta_2 + \beta_3 )*math$$

Now we can map the logistic regression output to these two equations. So we can say that the **coefficient for math is the effect of math when female = 0**.  More explicitly, we can say that for male students, a one-unit increase in math score yields a change in log odds of 0.13.  

On the other hand, for the female students, a one-unit increase in math score yields a change in log odds of $(.13 + .067) = 0.197$.  In terms of odds ratios, we can say that for male students, the odds ratio is $exp(.13)  = 1.14$ for a one-unit increase in math score and the odds ratio for female students is $exp(.197) = 1.22$ for a one-unit increase in math score.  **The ratio of these two odds ratios (female over male) turns out to be the exponentiated coefficient for the interaction term of female by math: $1.22/1.14 = exp(.067) = 1.07$.**

In [ ]:
np.exp(lr.params)